In [430]:
from sklearn import svm

In [431]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Test image
im_frame = Image.open("../Exercise3/positives/" + 'p01.png')
np_frame = np.array(im_frame.getdata())

# Load images (30, 573, 3) == (num_images, num_pixels, rgb)
positives = np.zeros((30, 576, 3))
for i in range(30):
    im_frame = Image.open("../Exercise3/positives/" + 'p' + ('0' if (i+1) < 10 else '') + str(i + 1) + '.png')
    positives[i] = np.array(im_frame.getdata())
    
negatives = np.zeros((30, 576, 3))
for i in range(30):
    im_frame = Image.open("../Exercise3/negatives/" + 'n' + ('0' if (i+1) < 10 else '') + str(i + 1) + '.png')
    negatives[i] = np.array(im_frame.getdata())

In [432]:
def get_RGB_means(images):
    return np.array([np.mean(img, axis=0) for i,img in enumerate(images)])

RGB_means_pos = get_RGB_means(positives)
RGB_means_neg = get_RGB_means(negatives)

In [433]:
def get_RGB_stds(images):
    return np.array([np.std(img, axis=0) for i,img in enumerate(images)])

RGB_stds_pos = get_RGB_stds(positives)
RGB_stds_neg = get_RGB_stds(negatives)

In [434]:
def get_top_five(sample):
    red = np.mean(np.sort(sample[:, :, 0], axis=1)[:, :5], axis=1)
    green = np.mean(np.sort(sample[:, :, 1], axis=1)[:, :5], axis=1)
    blue = np.mean(np.sort(sample[:, :, 2], axis=1)[:, :5], axis=1)
    return red, green, blue

top_red_pos, top_green_pos, top_blue_pos = get_top_five(positives)
top_red_neg, top_green_neg, top_blue_neg = get_top_five(negatives)

In [435]:
#samples_pos = np.column_stack((RGB_stds_pos, RGB_means_pos))
#samples_neg = np.column_stack((RGB_stds_neg, RGB_means_neg))

samples_pos = np.column_stack((RGB_stds_pos, top_red_pos, top_green_pos, top_blue_pos))
samples_neg = np.column_stack((RGB_stds_neg, top_red_neg, top_green_neg, top_blue_neg))

In [436]:
def split_data(no_of_tests, set_1, set_2):
    
    all_samples = np.concatenate((set_1, set_2), axis=0)
    
    n =  len(set_1)
    
    test_idxs = np.random.randint(0, n * 2, no_of_tests)
    
    test_features = all_samples[test_idxs,:]
    test_labels = (test_idxs >= 30)
    
    idxs_1 = test_idxs[test_idxs < n]
    idxs_2 = test_idxs[test_idxs >= n] - n
    
    train_1 = np.delete(set_1, idxs_1, axis=0)
    train_2 = np.delete(set_2, idxs_2, axis=0)

    train_features = np.concatenate((train_1, train_2), axis=0)
    train_labels = np.concatenate((np.array([0. for i in range(len(train_1))]), 
                                   np.array([1. for j in range(len(train_2))])
                                  ))
    
    return train_features, train_labels, test_features, test_labels

In [437]:
def train_and_test(no_of_tests, iterations, kernel='linear'):
    train_features, train_labels, test_features, test_labels = split_data(no_of_tests, samples_neg, samples_pos)

    clf = svm.SVC(kernel=kernel, max_iter=iterations)
    clf.fit(train_features, train_labels) 
    #print(linear_clf.coef_)
    #print(linear_clf.intercept_)
    return clf.score(test_features, test_labels)
    
    

In [438]:
iterations=10000
test_split_size=20
print("--Accuracies:--\nLinear: ", train_and_test(test_split_size,iterations,'linear'),
      "\nPolynomial: ", train_and_test(test_split_size,iterations,'poly'),
      "\nSigmoid: ", train_and_test(test_split_size,iterations,'sigmoid'),
      "\nRadial basis: ", train_and_test(test_split_size,iterations,'rbf')
     )

--Accuracies:--
Linear:  0.95 
Polynomial:  1.0 
Sigmoid:  0.25 
Radial basis:  0.5


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Ver